In [1]:
# https://scikit-learn.org/stable/datasets/index.html

# Importing the libraries
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split

# Importing the Boston Housing dataset
from sklearn.datasets import load_boston

In [2]:

# Loading the Boston Housing dataset
boston = load_boston()

In [3]:

# Initializing the dataframe
data = pd.DataFrame(boston.data)

In [4]:
#Adding the feature names to the dataframe
data.columns = boston.feature_names

In [5]:
#Adding target variable to dataframe
data['PRICE'] = boston.target
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
#Check the shape of dataframe
data.shape

(506, 14)

In [7]:
# Split the data into train and test with 80 train / 20 test
train, test = train_test_split(data, test_size=0.2, random_state = 1)
train, val = train_test_split(train, test_size=0.2, random_state = 1)

In [8]:
# Helper functions
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

def format_output(data):
    y1 = data.pop('PRICE')
    y1 = np.array(y1)
    y2 = data.pop('PTRATIO')
    y2 = np.array(y2)
    return y1, y2

In [9]:
# Get Y1 and Y2 as the 2 outputs and format them as np arrays
# PTRATIO - pupil-teacher ratio by town
train_stats = train.describe()
train_stats.pop('PRICE')
train_stats.pop('PTRATIO')
train_stats = train_stats.transpose()
train_Y = format_output(train)
test_Y = format_output(test)
val_Y = format_output(val)

In [10]:

# Normalize the training and test data
norm_train_X = np.array(norm(train))
norm_test_X = np.array(norm(test))
norm_val_X = np.array(norm(val))

In [24]:
print(len(norm_train_X[0]), len(train .columns))

12 12


In [21]:

def build_model():
    # Define model layers.
    input_layer = Input(shape=(len(train .columns),))
    first_dense = Dense(units='128', activation='relu')(input_layer)
    # Y1 output will be fed from the first dense
    y1_output = Dense(units='1', name='price_output')(first_dense)

    second_dense = Dense(units='128', activation='relu')(first_dense)
    # Y2 output will be fed from the second dense
    y2_output = Dense(units='1', name='ptratio_output')(second_dense)

    # Define the model with the input layer and a list of output layers
    model = Model(inputs=input_layer, outputs=[y1_output, y2_output])

    return model

In [22]:

model = build_model()

# Specify the optimizer, and compile the model with loss functions for both outputs
optimizer = tf.keras.optimizers.SGD(lr=0.001)
model.compile(optimizer=optimizer,
              loss={'price_output': 'mse', 'ptratio_output': 'mse'},
              metrics={'price_output': tf.keras.metrics.RootMeanSquaredError(),
                       'ptratio_output': tf.keras.metrics.RootMeanSquaredError()})

In [23]:
# Train the model for 200 epochs
history = model.fit(norm_train_X, train_Y,
                    epochs=100, batch_size=10, validation_data=(norm_test_X, test_Y))

ss: 13.0519 - price_output_loss: 11.6814 - ptratio_output_loss: 1.3705 - price_output_root_mean_squared_error: 3.4092 - ptratio_output_root_mean_squared_error: 1.1680 - val_loss: 14.2085 - val_price_output_loss: 11.6886 - val_ptratio_output_loss: 2.5198 - val_price_output_root_mean_squared_error: 3.4189 - val_ptratio_output_root_mean_squared_error: 1.5874
Epoch 55/100
33/33 [==============================] - 0s 4ms/step - loss: 11.0865 - price_output_loss: 9.4674 - ptratio_output_loss: 1.6191 - price_output_root_mean_squared_error: 3.0455 - ptratio_output_root_mean_squared_error: 1.2718 - val_loss: 15.0105 - val_price_output_loss: 12.4860 - val_ptratio_output_loss: 2.5245 - val_price_output_root_mean_squared_error: 3.5336 - val_ptratio_output_root_mean_squared_error: 1.5889
Epoch 56/100
33/33 [==============================] - 0s 3ms/step - loss: 16.2376 - price_output_loss: 14.7668 - ptratio_output_loss: 1.4708 - price_output_root_mean_squared_error: 3.7944 - ptratio_output_root_mean_